In [1]:
import numpy as np
import librosa
import cv2
import moviepy.editor as mpy

def create_particles(num_particles, width, height):
    particles = []
    for _ in range(num_particles):
        x = np.random.randint(0, width)
        y = np.random.randint(0, height)
        size = np.random.randint(2, 5)
        speed = np.random.randint(1, 5)
        particles.append([x, y, size, speed])
    return particles

def update_particles(particles, width, height, energy):
    for p in particles:
        p[1] -= p[3] * energy  # Move up with speed influenced by energy
        if p[1] < 0:
            p[1] = height
            p[0] = np.random.randint(0, width)
    return particles

def draw_particles(frame, particles, color):
    for p in particles:
        cv2.circle(frame, (int(p[0]), int(p[1])), p[2], color, -1)

def create_mosaic_mask(height, width, num_blocks=30, min_block_size=20, max_block_size=150):
    mask = np.ones((height, width), dtype=np.uint8) * 255

    for _ in range(num_blocks):
        block_size = np.random.randint(min_block_size, max_block_size)
        x = np.random.randint(0, width - block_size)
        y = np.random.randint(0, height - block_size)
        cv2.rectangle(mask, (x, y), (x + block_size, y + block_size), 0, -1)

    return mask


def apply_mosaic_effect(frame, mask, block_size=30):
    height, width = frame.shape[:2]
    small = cv2.resize(frame, (width // block_size, height // block_size))
    mosaic = cv2.resize(small, (width, height), interpolation=cv2.INTER_NEAREST)
    return np.where(mask[:,:,None] == 255, frame, mosaic)

def apply_blur_effect(frame, energy):
    blur_amount = int(energy * 2)  
    return cv2.GaussianBlur(frame, (blur_amount * 2 + 1, blur_amount * 2 + 1), 0)

def apply_bounce_effect(frame, current_onset, max_onset):
    bounce_amount = int(20 * current_onset / max_onset)  # 最大20ピクセルのバウンス
    if bounce_amount > 0:
        padded_frame = cv2.copyMakeBorder(frame, bounce_amount, bounce_amount, 0, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        return padded_frame[bounce_amount:-bounce_amount, :]
    else:
        return frame

def create_music_visualizer(image_path, audio_path, output_path):
    # Load the audio file
    y, sr = librosa.load(audio_path)

    # Compute the mel spectrogram and onset strength
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=64, fmax=8000)
    S_dB = librosa.power_to_db(S, ref=np.max)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)

    # Load the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Get image dimensions
    height, width, _ = img.shape

    # Create particles
    particles = create_particles(100, width, height)

    # Create a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('temp_output.mp4', fourcc, 30, (width, height))

    # Calculate frames per audio chunk
    chunk_size = sr // 30

    # Initialize variables for smooth animation
    prev_bars = np.zeros(64)
    zoom_factor = 1.0

    # Create mosaic mask with larger blocks randomly placed
    mosaic_mask = create_mosaic_mask(height, width)

    # Get max onset for normalization
    max_onset = np.max(onset_env)

    # Create the visualization
    for i in range(0, len(y), chunk_size):
        # Get the current audio chunk
        chunk = y[i:i+chunk_size]

        # Get the current spectrogram frame
        spec_frame = S_dB[:, i//chunk_size] if i//chunk_size < S_dB.shape[1] else S_dB[:, -1]

        # Get the current onset strength
        current_onset = onset_env[i//chunk_size] if i//chunk_size < len(onset_env) else onset_env[-1]

        # Calculate energy for particle movement and blur effect
        energy = np.mean(np.abs(chunk)) * 10

        # Create a copy of the image for this frame
        frame = img.copy()

        # Apply mosaic effect
        frame = apply_mosaic_effect(frame, mosaic_mask)

        # Update and draw particles
        particles = update_particles(particles, width, height, energy)
        draw_particles(frame, particles, (255, 255, 255))

        # Draw the bars
        bar_width = width // 64
        max_bar_height = height // 2
        for j, h in enumerate(spec_frame):
            # Smooth the bar height
            target_height = int(np.interp(h, [S_dB.min(), S_dB.max()], [0, max_bar_height]))
            prev_bars[j] = prev_bars[j] * 0.7 + target_height * 0.3
            bar_height = int(prev_bars[j])

            # Draw semi-transparent bar
            cv2.rectangle(frame, 
                          (j*bar_width, height), 
                          ((j+1)*bar_width, height - bar_height), 
                          (255, 255, 255), 
                          -1)

            # Draw bar outline
            cv2.rectangle(frame, 
                          (j*bar_width, height), 
                          ((j+1)*bar_width, height - bar_height), 
                          (200, 200, 200), 
                          2)

        # Apply alpha blending for transparency
        overlay = frame.copy()
        cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)

        # Apply blur effect based on energy
        frame = apply_blur_effect(frame, energy)

        # Apply a rhythmic zoom effect
        target_zoom = 1 + 0.05 * current_onset / max_onset
        zoom_factor = zoom_factor * 0.7 + target_zoom * 0.3
        scaled_frame = cv2.resize(frame, None, fx=zoom_factor, fy=zoom_factor)

        # Crop the scaled frame to original size
        start_y = (scaled_frame.shape[0] - height) // 2
        start_x = (scaled_frame.shape[1] - width) // 2
        frame = scaled_frame[start_y:start_y+height, start_x:start_x+width]

        # Apply bounce effect
        frame = apply_bounce_effect(frame, current_onset, max_onset)

        # Write the frame
        video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

    # Release the video writer
    video.release()

    # Add audio to the video using moviepy
    video_clip = mpy.VideoFileClip("temp_output.mp4")
    audio_clip = mpy.AudioFileClip(audio_path)
    final_clip = video_clip.set_audio(audio_clip)
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

    # Clean up temporary file
    import os
    os.remove("temp_output.mp4")

# Usage
image_path = 'image.png'
audio_path = 'audio.mp3'
output_path = 'output_video.mp4'

create_music_visualizer(image_path, audio_path, output_path)


Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
